## Min Cut Algorithm

The file kargerMinCut.txt contains the adjacency list representation of a simple undirected graph. There are 200 vertices labeled 1 to 200. The first column in the file represents the vertex label, and the particular row (other entries except the first column) tells all the vertices that the vertex is adjacent to. So for example, the 6th row looks like : "6	155	56	52	120	......". This just means that the vertex with label 6 is adjacent to (i.e., shares an edge with) the vertices with labels 155,56,52,120,......,etc

Your task is to code up and run the randomized contraction algorithm for the min cut problem and use it on the above graph to compute the min cut. (HINT: Note that you'll have to figure out an implementation of edge contractions. Initially, you might want to do this naively, creating a new graph from the old every time there's an edge contraction. But you should also think about more efficient implementations.) (WARNING: As per the video lectures, please make sure to run the algorithm many times with different random seeds, and remember the smallest cut that you ever find.) Write your numeric answer in the space provided. So e.g., if your answer is 5, just type 5 in the space provided.


In [177]:
import numpy as np
import copy
import tqdm


FILE = "kargerMinCut.txt"
SEP = "\t"

# FILE = "input_random_14_50.txt"
# SEP = " "

def get_edge_id(l,m):
    l = int(l)
    m = int(m)
    a,b = (l,m) if l<m else (m,l)

    return str(a)+"-"+str(b)

def get_edge_id_create_if_absent(edge_index, l,m):
    edge_id = get_edge_id(l,m)
    
    if edge_index not in index:
        l = len(index)
        index[edge_index] = l
        ret = l
    else:
        ret = index[edge_index]
        
    return ret

fp = open(FILE, "r+")


# Data Structures
min_cut_data = dict()  # Adjacancy List as a dictionary
min_cut_edges = []   # List of Edges
min_cut_edges_to_idx = {}  # Edge Id to List index
min_cut_idx_to_edge = {}  # Edge Id to List index

for line in (fp.readlines()):
    row = line.strip().split(SEP)
    min_cut_data[row[0]] = row[1:]

N = len(min_cut_data) # Number of Vertices

# Create edge list

for key, dt in min_cut_data.items():
    all_keys = min_cut_edges_to_idx.keys()
    for d in dt:
        edge_id = get_edge_id(key,d)
        if edge_id not in min_cut_edges_to_idx:
            l = len(min_cut_edges_to_idx)
            min_cut_edges_to_idx[edge_id] = l
            min_cut_edges.append(1)
            min_cut_idx_to_edge[l] = edge_id


print (np.sum(min_cut_edges))

 

2517


In [176]:
# Test

print ("Number of edges:", len(min_cut_edges))

# print (min_cut_idx_to_edge)
# print (min_cut_edges_to_idx)

Number of edges: 2517


In [130]:
def get_num_vertices(graph):
    return len(graph.items())

def remove_edge(graph, edges, edge):
    """
        let (u,v) = edge
        
        1) remove ("u-v") from the edge list
        2) copy all edges from v to u, except dups
        4) remove vertex v from graph
    """
    
    u,v = edge.split("-")
    #print ("u={},v={}".format(u, v))
    
    #1. Remove all edges v-u
    edges[get_edge_id(u,v)] = 0

    #2. Update adjacency lists of each k and v
    #print ("Remove", v, graph[v])
    for k in graph[v]:      
        
        if k == u:
            continue 
            
        #print (("neighbor {} of {}").format(k, v))
        
        # UPDATE GRAPH Structure: Remove v from the adj list of all its neighbours
        #print ("graph", k, graph[k])
        graph[k].remove(v)
        
        if u not in graph[k]:
            graph[k].append(u)
        # add the neighbor in the vertice of u
        if k not in graph[u]: 
            graph[u].append(k)

        # UPDATE EDGE STRUCTURE remove all edges from edge list
        edges.setdefault(get_edge_id(k,u), 0)
        edges[get_edge_id(k,u)] += edges[get_edge_id(v,k)]
        
        edges[get_edge_id(v,k)] = 0
        
           
    #3. Remove vertex v
    graph[u].remove(v)
    graph.pop(v)

def get_random_edge(edges):
        
    r = np.random.randint(0, get_num_edges(edges))
    
    i = 0
    for key, d in edges.items():
        if d == 0:
            continue
            
        if r - i <= d:
            return key
        
        i += d
        
    return key 

def get_num_edges(edges):
    N = 0
    for key, d in edges.items():
        N += d
        
    return N
    
def RandomContract(graph, edges):    
    while get_num_vertices(graph) > 2:
        edge = get_random_edge(edges)    
        #print ("Remove next edge", edge, "Num Vertices", get_num_vertices(graph))
        remove_edge(graph, edges, edge)


    return get_num_edges(edges)
    

In [129]:
# Example graph

"""
1   2
*---*
| / |
|/  |
*---*
3   4
"""
graph = {
    "1":["2", "3"],
    "2":["1", "3", "4"],
    "3":["1","2", "4"],
    "4":["2", "3"]
}
myedges = {
    "1-2":1,
    "1-3":1,
    "2-3":1,
    "3-4":1,
    "2-4":1
}
# edge =(get_random_edge(myedges))

edge = "3-1"
print ("REMOVE edge", edge)

print ("before",graph)
remove_edge(graph, myedges, edge )
print ("new graph:",graph)
print ("edges:",myedges)
print ("num_edges",get_num_edges(myedges))
print ("num_vertices",get_num_vertices(graph))

    
edge = "2-3"
print ("REMOVE edge", edge)

remove_edge(graph, myedges, edge )
print ("new graph:",graph)
print ("edges:",myedges)
print ("num_edges",get_num_edges(myedges))
print ("num_vertices",get_num_vertices(graph))



REMOVE edge 3-1
before {'2': ['1', '3', '4'], '4': ['2', '3'], '3': ['1', '2', '4'], '1': ['2', '3']}
new graph: {'2': ['3', '4'], '4': ['2', '3'], '3': ['2', '4']}
edges: {'2-3': 2, '1-2': 0, '1-3': 0, '3-4': 1, '2-4': 1}
num_edges 4
num_vertices 3
REMOVE edge 2-3
new graph: {'2': ['4'], '4': ['2']}
edges: {'2-3': 0, '1-2': 0, '1-3': 0, '3-4': 0, '2-4': 2}
num_edges 2
num_vertices 2


In [132]:
# Example graph
np.random.seed(2)
"""
1   2
*---*
| / |
|/  |
*---*
3   4
"""
graph = {
    "1":["2", "3"],
    "2":["1", "3", "4"],
    "3":["1","2", "4"],
    "4":["2", "3"]
}
myedges = {
    "1-2":1,
    "1-3":1,
    "2-3":1,
    "3-4":1,
    "2-4":1
}
# edge =(get_random_edge(myedges))

print (RandomContract(graph, myedges))
print ("new graph:",graph)
print ("edges:",myedges)
print ("num_edges",get_num_edges(myedges))
print ("num_vertices",get_num_vertices(graph))





2
new graph: {'2': ['1'], '1': ['2']}
edges: {'2-3': 0, '1-2': 2, '1-3': 0, '3-4': 0, '2-4': 0}
num_edges 2
num_vertices 2


In [94]:
np.random.seed(9)

graph = dict()
myedges = dict()

graph = copy.deepcopy(min_cut_data)
myedges = copy.deepcopy(min_cut_edges)

mincut = RandomContract(graph, myedges)
print ("Mincut", mincut)

Mincut 34


In [149]:
#print (list(edges.keys()))
#np.random.seed(1)

N = get_num_vertices(min_cut_data)
TOTAL = int(N**2  * np.log(N))
print ("TOTAL", TOTAL)

mincuts = []
new_min = 2*N

for i in tqdm.tqdm(range(TOTAL)):
    #np.random.seed(i)


    graph = {}
#     myedges = {}

    graph = copy.deepcopy(min_cut_data)
    myedges = copy.deepcopy(min_cut_edges)

    mincut = RandomContract(graph, myedges)
    #print ("Mincut", mincut, "seed", i)
    
    if mincut < new_min:
        print ("new min", mincut)
        new_min = mincut
        
    mincuts.append(mincut)
    
print (min(mincuts))


  0%|          | 1/211932 [00:00<8:44:39,  6.73it/s]

TOTAL 211932
new min 20



  0%|          | 12/211932 [00:01<10:13:22,  5.76it/s]

new min 17


  0%|          | 619/211932 [01:39<8:56:16,  6.57it/s] 

KeyboardInterrupt: 

In [98]:
print (i)

5949
